In [1]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from scipy.signal import argrelmin, argrelmax
import plotly.graph_objects as go

import yfinance as yf

C:\Users\edton\AppData\Local\Temp\ipykernel_43576\1124817892.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# min max plot


In [2]:
tdy = datetime.now()

past_days = 360
ago = datetime.now() - timedelta(days=past_days)
print(tdy)

2024-02-07 17:47:20.020997


In [7]:
ticker = "TSLA"
stock = yf.download(ticker, ago, tdy, progress=False)

c:\Users\edton\miniconda3\envs\fin_dashboard\Lib\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.



In [11]:
def merge_extrema_idx_with_val(max_idx: tuple, min_idx: tuple, stock_data: pd) -> np:
    """Merge and sort indices of max and min values from stock data along with their corresponding values."""
    max_values = stock_data["High"].iloc[max_idx]
    min_values = stock_data["Low"].iloc[min_idx]

    merged_idx = np.concatenate((max_idx, min_idx))
    merged_val = np.concatenate((max_values.values, min_values.values))
    # Representing min as 0 and max as 1
    merged_type = np.concatenate(([1] * len(max_idx), [0] * len(min_idx)))

    stacked_array = np.vstack((merged_idx, merged_val, merged_type))
    sorted_extrema = stacked_array[:, stacked_array[0].argsort()]

    return sorted_extrema


def clean_idx(max_idx: tuple, min_idx: tuple, stock_data: pd) -> tuple[list, list]:
    """Confirm that there are no consecutive minimums or maximums."""
    sorted_extrema = merge_extrema_idx_with_val(max_idx, min_idx, stock_data)

    prev = sorted_extrema[:, 0]
    clean_max_idx = []
    clean_min_idx = []

    for i in range(1, len(sorted_extrema[0])):
        curr = sorted_extrema[:, i]
        if curr[2] != prev[2]:
            idx = int(prev[0])
            clean_max_idx.append(idx) if prev[2] == 1 else clean_min_idx.append(idx)
            prev = curr
        # multi max
        elif curr[2] == 1:
            prev = prev if prev[1] > curr[1] else curr
        # multi min
        elif curr[2] == 0:
            prev = prev if prev[1] < curr[1] else curr

    # Add the final "prev" as it must be valid
    idx = int(prev[0])
    clean_max_idx.append(idx) if prev[2] == 1 else clean_min_idx.append(idx)

    return clean_max_idx, clean_min_idx


def eval_max_min(max_idx: tuple, min_idx: tuple, stock_data: pd) -> tuple[list, list]:
    """Evaluate the percentages of min/max difference from previous extremes"""
    sorted_extrema = merge_extrema_idx_with_val(max_idx, min_idx, stock_data)

    prev = sorted_extrema[:, 0]
    max_eval = []
    min_eval = []
    # Initialize the first local max or min
    max_eval.append(0) if prev[2] == 1 else min_eval.append(0)

    for i in range(1, len(sorted_extrema[0])):
        curr = sorted_extrema[:, i]
        eval = round((((curr[1] - prev[1]) / prev[1])) * 100, 1)
        (
            max_eval.append(f"{str(eval)}%")
            if eval > 0
            else min_eval.append(f"{str(eval)}%")
        )
        prev = curr

    return max_eval, min_eval

In [12]:
# https://plotly.com/python/text-and-annotations/
def tune_order(order, stock, feature="minmax"):
  if feature == "minmax":
    local_maxima_indices = argrelmax(stock['High'].values, order=order)[0]
    # local_maxima_values = stock['High'].iloc[local_maxima_indices]

    local_minima_indices = argrelmin(stock['Low'].values, order=order)[0]
    # local_minima_values = stock['Low'].iloc[local_minima_indices]
  else:
    local_maxima_indices = argrelmax(stock['Close'].values, order=order)[0]
    # local_maxima_values = stock['Close'].iloc[local_maxima_indices]

    local_minima_indices = argrelmin(stock['Close'].values, order=order)[0]
    # local_minima_values = stock['Close'].iloc[local_minima_indices]

  local_maxima_indices, local_minima_indices = clean_idx(local_maxima_indices, local_minima_indices, stock)

  # local_min_values = stock['Low'].iloc[local_minima_indices].values
  # txt = local_min_values.astype(int)
  # text_array = txt.astype(str)

  max_eval, min_eval = eval_max_min(local_maxima_indices, local_minima_indices, stock)


  # Create candlestick plot
  candlestick = go.Candlestick(x=stock.index,
                              open=stock["Open"],
                              high=stock["High"],
                              low=stock["Low"],
                              close=stock["Close"],
                              name="Candlestick")

  # Create scatter plots for local minima and maxima
  scatter_minima = go.Scatter(x=stock.index[local_minima_indices],
                              y=stock["Low"].iloc[local_minima_indices],
                              mode="markers+text",
                              marker=dict(color="red", size=8),
                              name="Local Minima",
                              text=min_eval,
                              textposition="bottom center")

  scatter_maxima = go.Scatter(x=stock.index[local_maxima_indices],
                              y=stock["High"].iloc[local_maxima_indices],
                              mode="markers+text",
                              marker=dict(color="green", size=8),
                              name="Local Maxima",
                              text=max_eval,
                              textposition="top center")


  # Create figure and add traces
  fig = go.Figure(data=[candlestick, scatter_minima, scatter_maxima])

  # Show the figure
  fig.update_layout(xaxis_rangeslider_visible=False)
  fig.show()

  return local_maxima_indices, local_minima_indices

In [13]:
i = 4
print(f"order = {i}, minmax")
local_max, local_min = tune_order(i, stock, feature="minmax")

order = 4, minmax
